In [ ]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
import os
os.chdir("..")
os.getcwd()

In [ ]:
# this file is excluded from the repo due to its size but can be reproduced by running EIA930API_Scriptv2.py
data = pd.read_csv("EIA_BAlims_2021-2023v2_full.csv") 

from_var = "fromferc"
to_var = "toferc"

data = data.loc[data[from_var] != data[to_var]] # remove intra-regional transfers
# Process the reported data so that fromferc = exporter and toferc = importer
data_exp = data.loc[data["value"] >= 0].copy()
data_exp["Reporter"] = data_exp[from_var]
data_imp = data.loc[data["value"] <= 0].copy()
data_imp["Reporter"] = data_imp[from_var]
data_imp.rename(columns={from_var:to_var,to_var:from_var},inplace=True)
data_imp["value"] *= -1

df = pd.concat([data_imp, data_exp], axis=0)
df["Combined"] = df[[from_var, to_var]].apply(tuple, 1).apply(sorted).apply(tuple)

## Plot flow duration curves

In [ ]:
interfaces = list(set(df["Combined"]))
percentile = 0.999 # change as desired to control for outlier data (an ESIG report used 99th percentile)

for (one,two) in interfaces:
    forward = df.loc[(df[from_var] == one) & (df[to_var] == two)]
    reverse = df.loc[(df[from_var] == two) & (df[to_var] == one)]
    try:
        forward_lim = forward.groupby([from_var, to_var])["value"].quantile(percentile, interpolation="lower").values[0]
    except:
        forward_lim = 1000000
    try:
        reverse_lim = reverse.groupby([from_var, to_var])["value"].quantile(percentile, interpolation="lower").values[0]
    except:
        reverse_lim = 1000000
    forward = forward[forward["value"]<forward_lim] # remove outliers above each interface's forward percentile
    reverse = reverse[reverse["value"]<reverse_lim] # remove outliers above each interface's reverse percentile
    ax = plt.axes()
    ax.plot(forward[forward["Reporter"]==one]["value"].sort_values(ascending=False).reset_index(drop=True), label=f"Forward (Reported by {one})",color="red")
    ax.plot(forward[forward["Reporter"]==two]["value"].sort_values(ascending=False).reset_index(drop=True), label=f"Forward (Reported by {two})",color="blue")
    ax.plot((reverse[reverse["Reporter"]==one]["value"].sort_values(ascending=False)*-1).reset_index(drop=True), label=f"Reverse (Reported by {one})",color="red",linestyle="dashed")
    ax.plot((reverse[reverse["Reporter"]==two]["value"].sort_values(ascending=False)*-1).reset_index(drop=True), label=f"Reverse (Reported by {two})",color="blue",linestyle="dashed")
    ax.set_xlim(left=0)
    ax.set_title(f"{one}-{two} Historical Flows (2021-2023)")
    ax.set_ylabel("MW Transfer")
    ax.set_xlabel("Number of Hours")
    ax.legend()
    plt.savefig(f"{one}-{two}_HistoricalDurationCurve.png")
    plt.close()
    

## Extract Maxes to produce csv of historical interface limits

Note the different methods for determining the "maxmimum" interface flow according to outlier removal and reporting entity.

In [ ]:
percentile = 0.99 # change as desired to control for outlier data (an ESIG report used 99th percentile)
df_max = df.groupby(["fromferc", "toferc","Reporter"])["value"].max().to_frame("Max")
df_max_avg = df_max.groupby(["fromferc", "toferc"])["Max"].mean().to_frame("Max_Avg") # finds average of the max flows reported by both regions
df_max_max = df_max.groupby(["fromferc", "toferc"])["Max"].max().to_frame("Max_Max") # takes the max of the max flows reported by both regions

df_99 = df.groupby(["fromferc", "toferc","Reporter"])["value"].quantile(percentile, interpolation="lower").to_frame(f"Q{percentile}")
df_99_avg = df_99.groupby(["fromferc", "toferc"])[f"Q{percentile}"].mean().to_frame(f"Q{percentile}_Avg") # finds average of the 99th % flows reported by both regions
df_99_max = df_99.groupby(["fromferc", "toferc"])[f"Q{percentile}"].max().to_frame(f"Q{percentile}_Max") # takes the max of the 99th % flows reported by both regions

df_hist = pd.concat([df_max_avg,df_max_max,df_99_avg,df_99_max], axis=1)#.reset_index()
#df_hist.to_csv("historicals_2021-2023.csv")